# US DFM Nowcasting Model

This Notebook contains a DFM mixed frequency nowcasting model for US Quarterly GDP in Python, using the latest edition of the [FRED MD database](https://research.stlouisfed.org/econ/mccracken/fred-databases/). The model is adapted from the one set up by [Chad Fulton in 2020](http://www.chadfulton.com/topics/statespace_large_dynamic_factor_models.html).

### Brief overview

The dynamic factor model considered in this notebook can be found in the `DynamicFactorMQ` class, which is a part of the time series analysis component (and in particular the state space models subcomponent) of Statsmodels. It can be accessed as follows:

```python
import statsmodels.api as sm
model = sm.tsa.DynamicFactorMQ(...)
```

**Data**

In this notebook, we'll use 127 monthly variables and 1 quarterly variable (GDP) from the [FRED-MD / FRED-QD dataset](https://research.stlouisfed.org/econ/mccracken/fred-databases/) (McCracken and Ng, 2016).

**Statistical model**:

The statistical model and the EM-algorithm used for parameter estimation are described in:

- Bańbura and Modugno (2014), "Maximum likelihood estimation of factor models on datasets with arbitrary pattern of missing data." ([Working paper](https://core.ac.uk/download/pdf/6684705.pdf), [Published](https://onlinelibrary.wiley.com/doi/full/10.1002/jae.2306?casa_token=tX0xS_49OXcAAAAA%3Aocw-egTRztTVg643NCHRCQUs_OGCPMTS78Qds4gk2nN6ViFjOMZYSDVip-0eeDwQCpvaTOTqjof5_wKI)), and
- Bańbura et al. (2011), "Nowcasting" ([Working paper](https://core.ac.uk/download/pdf/6518537.pdf), [Handbook chapter](https://www.oxfordhandbooks.com/view/10.1093/oxfordhb/9780195398649.001.0001/oxfordhb-9780195398649-e-8))

As in these papers, the basic specification starts from the typical "static form" of the dynamic factor model:

$$
\begin{aligned}
y_t & = \Lambda f_t + \epsilon_t \\
f_t & = A_1 f_{t-1} + \dots + A_p f_{t-p} + u_t
\end{aligned}
$$

The `DynamicFactorMQ` class allows for the generalizations of the model described in the references above, including:

- Limiting blocks of factors to only load on certain observed variables
- Monthly and quarterly mixed-frequency data, along the lines described by Mariano and Murasawa (2010)
- Allowing autocorrelation (via AR(1) specifications) in the idiosyncratic disturbances $\epsilon_t$
- Missing entries in the observed variables $y_t$

**Nowcasting, updating forecasts, and computing the "news"**

By including both monthly and quarterly variables, this model can be used to produce "nowcasts" of a quarterly variable before it is released, based on the monthly data for that quarter. For example, the advance estimate for the first quarter GDP is typically released in April, but this model could produce an estimate in March that was based on data through February.

Many forecasting and nowcasting exercises are updated frequently, in "real time", and it is therefore important that one can easily add new datapoints as they come in. As these new datapoints provide new information, the model's forecast / nowcast will change with new data, and it is also important that one can easily decompose these changes into contributions from each updated series to changes in the forecast.  Both of these steps are supported by all state space models in Statsmodels – including the `DynamicFactorMQ` model – as we show below.

**Other resources**

The [New York Fed Staff Nowcast](https://www.newyorkfed.org/research/policy/nowcast.html) is an application of this same dynamic factor model and (EM algorithm) estimation method. Although they use a different dataset, and update their results weekly, their underlying framework is the same as that used in this notebook.

For more details on the New York Fed Staff Nowcast model and results, see [Bok et al. (2018)](https://www.newyorkfed.org/research/staff_reports/sr830).

In [ ]:
import types
import numpy as np
import pandas as pd
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns

### Dataset

In this notebook, we estimate a dynamic factor model on a large panel of economic data released at a monthly frequency, along with GDP, which is only released at a quarterly frequency. The monthly datasets that we'll be using come from [FRED-MD database](https://research.stlouisfed.org/econ/mccracken/fred-databases/) (McCracken and Ng, 2016), and we will take real GDP from the companion FRED-QD database.

**Data vintage**

The FRED-MD dataset was launched in January 2015, and vintages are available for each month since then. The FRED-QD dataset was fully launched in May 2018, and monthly vintages are available for each month since then. 

**Data transformations**

The assumptions of the dynamic factor model in this notebook require that the factors and observed variables are stationary. However, this dataset contains raw economic series that clearly violate that assumptions – for example, many of them show distinct trends. As is typical in these exercises, we therefore transform the variables to induce stationarity. In particular, the FRED-MD and FRED-QD datasets include suggested transformations (coded 1-7, which typically apply differences or percentage change transformations) which we apply.

The exact details are in the `transform` function, below:

In [ ]:
def transform(column, transforms):
    transformation = transforms[column.name]
    # For quarterly data like GDP, we will compute
    # annualized percent changes
    mult = 4 if column.index.freqstr[0] == 'Q' else 1
    
    # 1 => No transformation
    if transformation == 1:
        pass
    # 2 => First difference
    elif transformation == 2:
        column = column.diff()
    # 3 => Second difference
    elif transformation == 3:
        column = column.diff().diff()
    # 4 => Log
    elif transformation == 4:
        column = np.log(column)
    # 5 => Log first difference, multiplied by 100
    #      (i.e. approximate percent change)
    #      with optional multiplier for annualization
    elif transformation == 5:
        column = np.log(column).diff() * 100 * mult
    # 6 => Log second difference, multiplied by 100
    #      with optional multiplier for annualization
    elif transformation == 6:
        column = np.log(column).diff().diff() * 100 * mult
    # 7 => Exact percent change, multiplied by 100
    #      with optional annualization
    elif transformation == 7:
        column = ((column / column.shift(1))**mult - 1.0) * 100
        
    return column


**Outliers**

Following McCracken and Ng (2016), we remove outliers (setting their value to missing), defined as observations that are more than 10 times the interquartile range from the series mean.

However, in this exercise we are interested in "nowcasting" real GDP growth for 2020Q2, which was greatly affected by economic shutdowns stemming from the COVID-19 pandemic. During the first half of 2020, there are a number of series which include extreme observations, many of which would be excluded by this outlier test. Because these observations are likely to be informative about real GDP in 2020Q2, we only remove outliers for the period 1959-01 through 2019-12.

The details of outlier removal are in the `remove_outliers` function, below:

In [ ]:
def remove_outliers(dta):
    # Compute the mean and interquartile range
    mean = dta.mean()
    iqr = dta.quantile([0.25, 0.75]).diff().T.iloc[:, 1]
    
    # Replace entries that are more than 10 times the IQR
    # away from the mean with NaN (denotes a missing entry)
    mask = np.abs(dta) > mean + 10 * iqr
    treated = dta.copy()
    treated[mask] = np.nan

    return treated

**Loading the data**

The `load_fredmd_data` function, below, performs the following actions, once for the FRED-MD dataset and once for the FRED-QD dataset:

1. Based on the `vintage` argument, it downloads a particular vintage of these datasets from the base URL https://files.stlouisfed.org/files/htdocs/fred-md into the `orig_[m|q]` variable.
2. Extracts the column describing which transformation to apply into the `transform_[m|q]` (and, for the quarterly dataset, also extracts the column describing which factor an earlier paper assigned each variable to).
3. Extracts the observation date (from the "sasdate" column) and uses it as the index of the dataset.
4. Applies the transformations from step (2).
5. Removes outliers for the period 1959-01 through 2019-12.

Finally, these are collected into an easy-to-use object (the `SimpleNamespace` object) and returned.

In [ ]:
def load_fredmd_data(vintage):
    base_url = 'https://files.stlouisfed.org/files/htdocs/fred-md'
    
    # - FRED-MD --------------------------------------------------------------
    # 1. Download data
    orig_m = (pd.read_csv(f'{base_url}/monthly/{vintage}.csv')
                .dropna(how='all'))
    
    # 2. Extract transformation information
    transform_m = orig_m.iloc[0, 1:]
    orig_m = orig_m.iloc[1:]

    # 3. Extract the date as an index
    orig_m.index = pd.PeriodIndex(orig_m.sasdate.tolist(), freq='M')
    orig_m.drop('sasdate', axis=1, inplace=True)

    # 4. Apply the transformations
    dta_m = orig_m.apply(transform, axis=0,
                         transforms=transform_m)

    # 5. Remove outliers (but not in 2020)
    dta_m.loc[:'2019-12'] = remove_outliers(dta_m.loc[:'2019-12'])

    # - FRED-QD --------------------------------------------------------------
    # 1. Download data
    orig_q = (pd.read_csv(f'{base_url}/quarterly/{vintage}.csv')
                .dropna(how='all'))

    # 2. Extract factors and transformation information
    factors_q = orig_q.iloc[0, 1:]
    transform_q = orig_q.iloc[1, 1:]
    orig_q = orig_q.iloc[2:]

    # 3. Extract the date as an index
    orig_q.index = pd.PeriodIndex(orig_q.sasdate.tolist(), freq='Q')
    orig_q.drop('sasdate', axis=1, inplace=True)

    # 4. Apply the transformations
    dta_q = orig_q.apply(transform, axis=0,
                          transforms=transform_q)

    # 5. Remove outliers (but not in 2020)
    dta_q.loc[:'2019Q4'] = remove_outliers(dta_q.loc[:'2019Q4'])
    
    # - Output datasets ------------------------------------------------------
    return types.SimpleNamespace(
        orig_m=orig_m, orig_q=orig_q,
        dta_m=dta_m, transform_m=transform_m,
        dta_q=dta_q, transform_q=transform_q, factors_q=factors_q)

In [ ]:
# Load the current dataset
data = load_fredmd_data("current")

In [ ]:
# Definitions from the Appendix for FRED-MD variables
defn_m = pd.read_csv('../data/FRED/fredmd_definitions.csv', encoding_errors='ignore')
defn_m.index = defn_m.fred
defn_m = defn_m.loc[data.dta_m.columns.intersection(defn_m.fred), :]
map_m = defn_m['description'].to_dict()

# Definitions from the Appendix for FRED-QD variables
defn_q = pd.read_csv('../data/FRED/fredqd_definitions.csv', encoding_errors='ignore')
defn_q.index = defn_q.fred
defn_q = defn_q.loc[data.dta_q.columns.intersection(defn_q.fred), :]
map_q = defn_q['description'].to_dict()

# Example of the information in these files:
defn_m.head()

In [ ]:
# Plotting GDP
data.orig_q.GDPC1.plot(title = map_q["GDPC1"], figsize = (12, 5))

In [ ]:
data.dta_q.GDPC1.plot(title = "Real GDP Growth (%)", figsize = (12, 5))

To aid interpretation of the results, we'll replace the names of our dataset with the "description" field.

In [ ]:
# Replace the names of the columns in each monthly and quarterly dataset
data.orig_m = data.orig_m[map_m.keys()].rename(columns = map_m)
data.dta_m = data.dta_m[map_m.keys()].rename(columns = map_m)
data.orig_q = data.orig_q[map_q.keys()].rename(columns = map_q)
data.dta_q = data.dta_q[map_q.keys()].rename(columns = map_q)

**Data groups**

Below, we get the groups for each series from the definition files above, and then show how many of the series that we'll be using fall into each of the groups.

We'll also re-order the series by group, to make it easier to interpret the results.

Since we're including the quarterly real GDP variable in our analysis, we need to assign it to one of the groups in the monthly dataset. It fits best in the "Output and income" group.

In [ ]:
# Get the mapping of variable id to group name, for monthly variables
groups = defn_m[['description', 'group']].copy()

# Re-order the variables according to the definition CSV file
# (which is ordered by group)
columns = [name for name in defn_m['description'] if name in data.dta_m.columns]

# Add real GDP (our quarterly variable) into the "Output and Income" group
gdp_description = defn_q.loc['GDPC1', 'description']
groups = pd.concat([groups, pd.DataFrame([{'description': gdp_description, 'group': 'Output and Income'}])], 
                   ignore_index=True)

# Display the number of variables in each group
(groups.groupby('group', sort=False)
       .count()
       .rename({'description': '# series in group'}, axis=1))

In [ ]:
# Saving Data
data.orig_m.to_csv("../data/FRED/monthly.csv")
data.dta_m.to_csv("../data/FRED/monthly_transformed.csv")
data.orig_q.to_csv("../data/FRED/quarterly.csv")
data.dta_q.to_csv("../data/FRED/quarterly_transformed.csv")
groups.to_csv("../data/FRED/groups.csv")

### Model specification

**Factor specification**

Based on screeplots calculated for the overall dataset and within each group, we opt for 2 global factors and 1-2 factors for the different groups, which evolve by (vector) autoregressive processes of order 1-4:

In [ ]:
# Construct the variable => list of factors dictionary
factors = {row['description']: ['Global', row['group']]
           for ix, row in groups.iterrows()}

# Check that we have the desired output for "Real personal income"
print(factors['Real Personal Income'])

**Factor multiplicities**

The `factor_multiplicities` argument defaults to `1`, but it can be passed a dictionary with keys equal to factor names (from the `factors` argument) and values equal to an integer. Note that the default for each factor is 1, so we only include in this dictionary factors that have multiplicity greater than 1.

In [ ]:
factor_multiplicities = {'Global': 2, 
                         'Consumption, Orders, and Inventories': 2, 
                         'Interest and Exchange Rates': 2}

**Factor orders**

Finally, we need to specify the lag order of the (vector) autoregressions that govern the dynamics of the factors. This is done via the `factor_orders` argument. The `factor_orders` argument also defaults to `1`,

In [ ]:
factor_orders = {'Global': 4, 
                 'Consumption, Orders, and Inventories': 4, 
                 'Housing': 2,
                 'Interest and Exchange Rates': 3,
                 'Money and Credit': 2,
                 'Output and Income': 2}

**Creating the model**

Given the factor specification, above, we can finish the model specification and create the model object.

The `DynamicFactorMQ` model class has the following primary arguments:

1. `endog` and `endog_quarterly`

   These arguments are used to pass the observed variables to the model. There are two ways to provide the data:
   
   1. If you are specifying a monthly / quarterly mixed frequency model, then you would pass the monthly data to `endog` and the quarterly data to the keyword argument `endog_quarterly`. This is what we have done below.
   2. If you are specifying any other kind of model, then you simply pass all of your observed data to the `endog` variable and you do not include the `endog_quarterly` argument. In this case, the `endog` data does not need to be monthly - it can be any frequency (or no frequency).


2. `factors`, `factor_orders`, and `factor_multiplicities`

   These arguments were described above.


3. `idiosyncratic_ar1`

   As noted in the "Brief Overview" section, above, the `DynamicFactorMQ` model allows the idiosyncratic disturbance terms to be modeled as independent AR(1) processes or as iid variables. The default is `idiosyncratic_ar1=True`, which can be useful in modeling some of the idiosyncratic serial correlation, for example for forecasting.


4. `standardize`

   Although we earlier transformed all of the variables to be stationary, they will still fluctuate around different means and they may have very different scales. This can make it difficult to fit the model. In most applications, therefore, the variables are standardized by subtracting the mean and dividing by the standard deviation. This is the default, so we do not set this argument below.

   It is recommended for users to use this argument rather than standardizing the variables themselves. This is because if the `standardize` argument is used, then the model will automatically reverse the standardization for all post-estimation results like prediction, forecasting, and computation of the impacts of the "news". This means that these results can be directly compared to the input data.
   
**Note**: to generate the best nowcasts for GDP growth in 2020Q2, we will restrict the sample to start in 2000-01 rather than in 1960 (for example, to guard against the possibility of structural changes in underlying parameters).

In [ ]:
# Get the baseline monthly and quarterly datasets
start = '2000'
endog_m = data.dta_m.loc[start:, :]
gdp_description = defn_q.loc['GDPC1', 'description']
endog_q = data.dta_q.loc[start:, [gdp_description]]

# Construct the dynamic factor model
model = sm.tsa.DynamicFactorMQ(
    endog_m, endog_quarterly=endog_q,
    factors=factors, factor_orders=factor_orders,
    factor_multiplicities=factor_multiplicities)

# Another model without separate blocks but 9 global factors
model_global = sm.tsa.DynamicFactorMQ(
    endog_m, endog_quarterly=endog_q,
    factors = 1, # {x: 'Global' for x in list(groups.description)},
    factor_orders = 4, # {'Global': 4},
    factor_multiplicities= 9) #{'Global': 9})    

In [ ]:
# model.summary()
model_global.summary()

In [ ]:
# Fit blocked model
results = model.fit(disp=10)

In [ ]:
# Fit global model
results_global = model_global.fit(disp=10)

In [ ]:
results.summary()

In [ ]:
results_global.summary()

### Estimated factors

In addition to the estimates of the parameters, the `results` object contains the estimates of the latent factors. These are most conveniently accessed through the `factors` attribute. This attribute in turn contains four sub-attributes:

- `smoothed`: estimates of the factors, conditional on the full dataset (also called "smoothed" or "two-sided" estimates)
- `smoothed_cov`: covariance matrix of the factor estimates, conditional on the full dataset
- `filtered`: estimates of the factors, where the estimate at time $t$ only uses information through time $t$ (also called "filtered" or "one-sided" estimates
- `filtered_cov`: covariance matrix of the factor estimates, where the estimate at time $t$ only uses information through time $t$

As an example, in the next cell we plot three of the smoothed factors and 95% confidence intervals.

**Note**: The estimated factors are not identified without additional assumptions that this model does not impose (see for example Bańbura and Modugno, 2014, for details). As a result, it can be difficult to interpet the factors themselves. (Despite this, the space spanned by the factors *is* identified, so that forecasting and nowcasting exercises, like those we discuss later, are unambiguous).

For example, in the plot
below, the "Global.1" factor increases markedly in 2009, following the global financial crisis. However, many of the factor loadings in the summary above are negative – for example, this is true of the output, consumption, and income series. Therefore the increase in the "Global.1" factor during this period actually implies a strong *decrease* in output, consumption, and income.

In [ ]:
# [x for x in dir(results) if not x.startswith("_")]

In [ ]:
def plot_factors(results, factor_names):
    # Get estimates of the global and labor market factors,
    # conditional on the full dataset ("smoothed")
    mean = results.factors.smoothed[factor_names]

    # Compute 95% confidence intervals
    from scipy.stats import norm
    std = pd.concat([results.factors.smoothed_cov.loc[name, name]
                    for name in factor_names], axis=1)
    crit = norm.ppf(1 - 0.05 / 2)
    lower = mean - crit * std
    upper = mean + crit * std

    with sns.color_palette('deep'):
        fig, ax = plt.subplots(figsize=(14, 6))
        mean.plot(ax=ax)
        
        for name in factor_names:
            ax.fill_between(mean.index, lower[name], upper[name], alpha=0.3)
        
        ax.set(title='Estimated factors: smoothed estimates and 95% confidence intervals')
        fig.tight_layout();

In [ ]:
factor_names = results.factors.smoothed.columns.to_list() # ['Global.1', 'Global.2', 'Labor Market']
plot_factors(results, factor_names)

In [ ]:
plot_factors(results_global, results_global.factors.smoothed.columns.to_list())

#### Explanatory power of the factors

One way to examine how the factors relate to the observed variables is to compute the explanatory power that each factor has for each variable, by regressing each variable on a constant plus one or more of the smoothed factor estimates and storing the resulting $R^2$, or "coefficient of determination", value.

**Computing $R^2$**

The `get_coefficients_of_determination` method in the results object has three options for the `method` argument:

- `method='individual'` retrieves the $R^2$ value for each observed variable regressed on each individual factor (plus a constant term)
- `method='joint'` retrieves the $R^2$ value for each observed variable regressed on all factors that the variable loads on
- `method='cumulative'` retrieves the $R^2$ value for each observed variable regressed on an expanding set of factors. The expanding set begins with the $R^2$ from a regression of each variable on the first factor that the variable loads on (as it appears in, for example, the summary tables above) plus a constant. For the next factor in the list, the $R^2$ is computed by a regression on the first two factors (assuming that a given variable loads on both factors).

**Example:** top 10 variables explained by the global factors

Below, we compute according to the `method='individual'` approach, and then show the top 10 observed variables that are explained (individually) by each of the two global factors.

In [ ]:
rsquared = results.get_coefficients_of_determination(method='individual')
factor_names = ['Global.1', 'Global.2']

top_ten = []
for factor_name in factor_names:
    top_factor = (rsquared[factor_name].sort_values(ascending=False)
                                       .iloc[:10].round(2).reset_index())
    top_factor.columns = pd.MultiIndex.from_product([
        [f'Top ten variables explained by {factor_name}'],
        ['Variable', r'$R^2$']])
    top_ten.append(top_factor)
pd.concat(top_ten, axis=1)

**Plotting $R^2$**

When there are a large number of observed variables, it is often easier to plot the $R^2$ values for each variable. This can be done using the `plot_coefficients_of_determination` method in the results object. It accepts the same `method` arguments as the `get_coefficients_of_determination` method, above.

Below, we plot the $R^2$ values from the "individual" regressions, for each factor. Because there are so many variables, this graphical tool is best for identifying trends overall and within groups, and we do not display the names of the variables on the x-axis label.

In [ ]:
with sns.color_palette('deep'):
    fig = results.plot_coefficients_of_determination(method='individual', figsize=(14, 12))
    fig.suptitle(r'$R^2$ - regression on individual factors', fontsize=14, fontweight=600)
    fig.tight_layout(rect=[0, 0, 1, 0.95]);

In [ ]:
with sns.color_palette('deep'):
    fig = results_global.plot_coefficients_of_determination(method='individual', figsize=(14, 12))
    fig.suptitle(r'$R^2$ - regression on individual factors', fontsize=14, fontweight=600)
    fig.tight_layout(rect=[0, 0, 1, 0.95]);

Alternatively, we might look at the overall explanatory value to a given variable of all factors that the variable loads on. To do that, we can use the same function but with `method='joint'` .

To make it easier to identify patterns, we add in shading and labels to identify the different groups of variables, as well as our only quarterly variable, GDP.

In [ ]:
def plot_r2_joint(results):
    group_counts = defn_m[['description', 'group']]
    group_counts = group_counts[group_counts['description'].isin(data.dta_m.columns)]
    group_counts = group_counts.groupby('group', sort=False).count()['description'].cumsum()

    with sns.color_palette('deep'):
        fig = results.plot_coefficients_of_determination(method='joint', figsize=(14, 3));

        # Add in group labels
        ax = fig.axes[0]
        ax.set_ylim(0, 1.2)
        for i in np.arange(1, len(group_counts), 2):
            start = 0 if i == 0 else group_counts[i - 1]
            end = group_counts[i] + 1
            ax.fill_between(np.arange(start, end) - 0.6, 0, 1.2, color='k', alpha=0.1)
        for i in range(len(group_counts)):
            start = 0 if i == 0 else group_counts[i - 1]
            end = group_counts[i]
            n = end - start
            text = group_counts.index[i]
            if len(text) > n:
                text = text[:n - 3] + '...'

            ax.annotate(text, (start + n / 2, 1.1), ha='center')

        # Add label for GDP
        ax.set_xlim(-1.5, model.k_endog + 0.5)
        ax.annotate('GDP', (model.k_endog - 1.1, 1.05), ha='left', rotation=90)

        fig.tight_layout();

In [ ]:
plot_r2_joint(results)

In [ ]:
plot_r2_joint(results_global)

### Forecasting

One of the benefits of these models is that we can use the dynamics of the factors to produce forecasts of any of the observed variables. This is straightforward here, using the `forecast` or `get_forecast` results methods. These take a single argument, which must be either:

- an integer, specifying the number of steps ahead to forecast
- a date, specifying the date of the final forecast to make

The `forecast` method only produces a series of point forecasts for all of the observed variables, while the `get_forecast` method returns a new forecast results object, that can also be used to compute confidence intervals. 

**Note**: these forecasts are in the same scale as the variables passed to the `DynamicFactorMQ` constructor, even if `standardize=True` has been used.

Below is an example of the `forecast` method.

In [ ]:
# Create point forecasts, 3 steps ahead
point_forecasts = results.forecast(steps=3)

# Print the forecasts for the first 5 observed variables
print(point_forecasts.T.head())

In addition to `forecast` and `get_forecast`, there are two more general methods, `predict` and `get_prediction` that allow for both of in-sample prediction and out-of-sample forecasting. Instead of a `steps` argument, they take `start` and `end` arguments, which can be either in-sample dates or out-of-sample dates.

Below, we give an example of using `get_prediction` to show in-sample predictions and out-of-sample forecasts for some spreads between Treasury securities and the Federal Funds Rate.

In [ ]:
# Create forecasts results objects, through the end of 2024
prediction_results = results.get_prediction(start='2000', end='2024')
prediction_results_global = results_global.get_prediction(start='2000', end='2024')

def plot_prediction(prediction_results, variables, start = '2000', end = '2024'):

    # The `predicted_mean` attribute gives the same
    # point forecasts that would have been returned from
    # using the `predict` or `forecast` methods.
    point_predictions = prediction_results.predicted_mean[variables]

    # We can use the `conf_int` method to get confidence
    # intervals; here, the 95% confidence interval
    ci = prediction_results.conf_int(alpha=0.05)
    lower = ci[[f'lower {name}' for name in variables]]
    upper = ci[[f'upper {name}' for name in variables]]

    latest = str(data.orig_m.index[-1])

    # Plot the forecasts and confidence intervals
    with sns.color_palette('deep'):
        fig, ax = plt.subplots(figsize=(14, 4))

        # Plot the in-sample predictions
        point_predictions.loc[:latest].plot(ax=ax)

        # Plot the out-of-sample forecasts
        point_predictions.loc[latest:].plot(ax=ax, linestyle='--',
                                            color=['C0', 'C1', 'C2'],
                                            legend=False)

        # Confidence intervals
        for name in variables:
            ax.fill_between(ci.index,
                            lower[f'lower {name}'],
                            upper[f'upper {name}'], alpha=0.1)
            
        # Forecast period, set title
        ylim = ax.get_ylim()
        ax.vlines(latest, ylim[0], ylim[1], linewidth=1)
        ax.annotate(r' Forecast $\rightarrow$', (latest, -1.7))
        ax.set(title='In-sample predictions and out-of-sample forecasts, with 95% confidence intervals', ylim=ylim)
        
        fig.tight_layout()

In [ ]:
variables = ['1-Year Treasury C Minus FEDFUNDS',
             '5-Year Treasury C Minus FEDFUNDS',
             '10-Year Treasury C Minus FEDFUNDS']
             
plot_prediction(prediction_results_global, variables)

In [ ]:
variables = ['Real Gross Domestic Product, 3 Decimal (Billions of Chained 2012 Dollars)'] 
             # 'Real Personal Income', 'Real personal consumption expenditures']

plot_prediction(prediction_results_global, variables)

#### Forecasting example

The variables that we showed in the forecasts above were not transformed from their original values. As a result, the predictions were already interpretable as spreads. For the other observed variables that were transformed prior to construting the model, our forecasts will be in the transformed scale.

For example, although the original data in the FRED-MD/QD datasets for Real GDP is in "Billions of Chained 2012 Dollars", this variable was transformed to the annualized quarterly growth rate (percent change) for inclusion in the model. Similarly, the Civilian Unemployment Rate was originally in "Percent", but it was transformed into the 1-month change (first difference) for inclusion in the model.

Because the transformed data was provided to the model, the prediction and forecasting methods will produce predictions and forecasts in the transformed space. (Reminder: the transformation step, which we did prior to constructing the model, is different from the standardization step, which the model handles automatically, and which we do not need to manually reverse).

Below, we compute and plot the forecasts directly from the model associated with real GDP and the unemployment rate.

In [ ]:
[x for x in dir(prediction_results) if not x.startswith("_")]

In [ ]:
# Get the titles of the variables as they appear in the dataset
unemp_description = 'Civilian Unemployment Rate'
gdp_description = 'Real Gross Domestic Product, 3 Decimal (Billions of Chained 2012 Dollars)'

latest = str(data.orig_m.index[-1])

# Compute the point forecasts
point_fcst = prediction_results.predicted_mean.loc['2023-01':, :]
fcast_m = point_fcst[unemp_description]
fcast_q = point_fcst[gdp_description].resample('Q').last()

# For more convenient plotting, combine the observed data with the forecasts
plot_m = pd.concat([data.dta_m.loc['2000':, unemp_description], fcast_m])
plot_q = pd.concat([data.dta_q.loc['2000':, gdp_description], fcast_q])

In [ ]:
with sns.color_palette('deep'):
    fig, axes = plt.subplots(2, figsize=(14, 7))

    # Plot real GDP growth, data and forecasts
    plot_q.plot(ax=axes[0])
    axes[0].set(title='Real Gross Domestic Product (transformed: annualized growth rate)')
    axes[0].hlines(0, plot_q.index[0], plot_q.index[-1], linewidth=1)

    # Plot the change in the unemployment rate, data and forecasts
    plot_m.plot(ax=axes[1])
    axes[1].set(title='Civilian Unemployment Rate (transformed: change)')
    axes[1].hlines(0, plot_m.index[0], plot_m.index[-1], linewidth=1)

    # Show the forecast period in each graph
    for i in range(2):
        ylim = axes[i].get_ylim()
        axes[i].fill_between(plot_q.loc['2023-01':].index,
                             ylim[0], ylim[1], alpha=0.1, color='C0')
        axes[i].annotate(r' Forecast $\rightarrow$',
                         ('2022-08', ylim[0] + 0.1 * ylim[1]))
        axes[i].set_ylim(ylim)

    # Title
    fig.suptitle('Data and forecasts (February 2020 vintage), transformed scale',
                 fontsize=14, fontweight=600)

    fig.tight_layout(rect=[0, 0, 1, 0.95]);

In [ ]:
# Reverse the transformations

# For real GDP, we take the level in 2000Q1 from the original data,
# and then apply the growth rates to compute the remaining levels
plot_q_orig = (plot_q / 100 + 1)**0.25
plot_q_orig.loc['2000Q1'] = data.orig_q.loc['2000Q1', gdp_description]
plot_q_orig = plot_q_orig.cumprod()

# For the unemployment rate, we take the level in 2000-01 from
# the original data, and then we apply the changes to compute the
# remaining levels
plot_m_orig = plot_m.copy()
plot_m_orig.loc['2000-01'] = data.orig_m.loc['2000-01', unemp_description]
plot_m_orig = plot_m_orig.cumsum()

In [ ]:
with sns.color_palette('deep'):
    fig, axes = plt.subplots(2, figsize=(14, 7))

    # Plot real GDP, data and forecasts
    plot_q_orig.plot(ax=axes[0])
    axes[0].set(title=('Real Gross Domestic Product'
                       ' (original scale: Billions of Chained 2012 Dollars)'))

    # Plot the unemployment rate, data and forecasts
    plot_m_orig.plot(ax=axes[1])
    axes[1].set(title='Civilian Unemployment Rate (original scale: Percent)')

    # Show the forecast period in each graph
    for i in range(2):
        ylim = axes[i].get_ylim()
        axes[i].fill_between(plot_q.loc['2023-01':].index,
                             ylim[0], ylim[1], alpha=0.1, color='C0')
        axes[i].annotate(r' Forecast $\rightarrow$',
                         ('2022-08', ylim[0] + 0.5 * (ylim[1] - ylim[0])))
        axes[i].set_ylim(ylim)

    # Title
    fig.suptitle('Data and forecasts (February 2020 vintage), original scale',
                 fontsize=14, fontweight=600)

    fig.tight_layout(rect=[0, 0, 1, 0.95]);